In [66]:
import csv, os, copy
from datetime import datetime, timedelta

gt_directory_path = "2025_google_transit"

GT_TRAM_STOP_TYPE = "0"

import math

def haversine_distance(coord1, coord2):
    R = 6371.0
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def load_gt_file(file_name):
    file_path = os.path.join(gt_directory_path, file_name)
    with open(file_path, "r", encoding="utf-8-sig") as file:
        reader = csv.reader(file, delimiter=",", quotechar='"')
        headers = next(reader)
        exported_rows = []
        for row in reader:
            exported_row = {}
            for header, item in zip(headers, row):
                exported_row[header] = item
            exported_rows.append(exported_row)
        return exported_rows
    
def gt_get_first(data_rows, field_name, value):
    for row in data_rows:
        if row[field_name] == value:
            return row
    raise Exception(f"Row not found with {field_name=} {value=}")

def gt_has_row(data_rows, field_name, value):
    for row in data_rows:
        if row[field_name] == value:
            return True
    return False

def gt_make_1_1_map(data_rows, id_field_name, force=False):
    data_map = {}
    for row in data_rows:
        row_id = row[id_field_name]
        if row_id in data_map and not force:
            raise Exception("multiple rows with same id found")
        data_map[row_id] = row
    return data_map

def gt_make_1_many_map(data_rows, id_field_name):
    data_map = {}
    for row in data_rows:
        row_id = row[id_field_name]
        if row_id in data_map:
            data_map[row_id].append(row)
        else:
            data_map[row_id] = [row]
    return data_map

def gt_get_all(data_rows, field_name, value):
    return [
        row for row in data_rows
        if row[field_name] == value
    ]

def gt_parse_date(date_str):
    year = int(date_str[0:4])
    month = int(date_str[4:6])
    day = int(date_str[6:8])
    return datetime(year, month, day)

def gt_parse_time(time_str, date_str: str):
    h, m, s = [int(i) for i in time_str.split(":")]
    date = gt_parse_date(date_str)
    if h >= 24:
        h -= 24
        date += timedelta(days=1)
    return datetime(date.year, date.month, date.day, h, m, s)

def gt_date_strs_between(start_date_str, end_date_str, test_date_str):
    start_date = gt_parse_date(start_date_str)
    end_date = gt_parse_date(end_date_str)
    test_date = gt_parse_date(test_date_str)
    return gt_dates_between(start_date, end_date, test_date)

def gt_dates_between(start_date, end_date, test_date):
    return (start_date <= test_date) and (test_date <= end_date)

gt_routes = load_gt_file("routes.txt")
gt_transfers = load_gt_file("transfers.txt")
gt_stops = load_gt_file("stops.txt")
gt_trips = load_gt_file("trips.txt")
gt_stop_times = load_gt_file("stop_times.txt")
gt_calendar = load_gt_file("calendar.txt")
gt_calendar_dates = load_gt_file("calendar_dates.txt")

gt_stops_map = gt_make_1_1_map(gt_stops, "stop_id")
gt_stop_times_map = gt_make_1_many_map(gt_stop_times, "trip_id")
gt_calendar_map = gt_make_1_1_map(gt_calendar, "service_id")
gt_calendar_dates_map = gt_make_1_many_map(gt_calendar_dates, "service_id")
gt_route_map = gt_make_1_1_map(gt_routes, "route_short_name")

GT_WEEKDAY_NAMES = [
    "monday", "tuesday", "wednesday",
    "thursday", "friday", "saturday", "sunday"
]

def is_trip_available(service_id, date_str):
    lookup_date = gt_parse_date(date_str)
    calendar = gt_calendar_map[service_id]
    start_date = gt_parse_date(calendar["start_date"])
    end_date = gt_parse_date(calendar["end_date"])
    if not gt_dates_between(start_date, end_date, lookup_date):
        return False
    
    weekday_name = GT_WEEKDAY_NAMES[lookup_date.weekday()]
    normally_available = calendar[weekday_name] == "1"

    if service_id in gt_calendar_dates_map:
        exception_calendar_dates = gt_calendar_dates_map[service_id]
        for calendar_date in exception_calendar_dates:
            if calendar_date != date_str:
                continue
            exception_type = calendar_date["exception_type"]
            if exception_type == "1":
                return True # service added
            elif exception_type == "2":
                return False # service removed
        
    return normally_available

def

In [67]:
class TramConnection:

    def __init__(self, stops=None, stop_arrival_times=None, stop_departure_times=None, transportation_names=None):
        self.stops = [] if stops is None else stops
        self.stop_arrival_times = [] if stop_arrival_times is None else stop_arrival_times
        self.stop_departure_times = [] if stop_departure_times is None else stop_departure_times
        self.transportation_names = [] if transportation_names is None else transportation_names
        
    def __str__(self):
        out_str = ""
        for stop, ari, dep, tra in self.zip():
            ari = f"[{ari}]" if ari is not None else ""
            tra = f"({tra})" if tra is not None else ""
            dep = f"[{dep}]" if dep is not None else ""
            out_str += f"{ari} {stop.name:30} {tra} {dep}\n"
        return out_str

    def zip(self):
        return zip(self.stops, self.stop_arrival_times, self.stop_departure_times, self.transportation_names)

    def add_stop(self, stop, arrival_time, departure_time, transportation_name):
        self.stops.append(stop)
        self.stop_arrival_times.append(arrival_time)
        self.stop_departure_times.append(departure_time)
        self.transportation_names.append(transportation_name)

    def remove_last(self):
        self.stops.pop()
        self.stop_arrival_times.pop()
        self.stop_departure_times.pop()
        self.transportation_names.pop()
    
    def slice(self, index) -> "TramConnection":
        copy = TramConnection()
        copy.stops = self.stops[index:]
        copy.stop_arrival_times = self.stop_arrival_times[index:]
        copy.stop_departure_times = self.stop_departure_times[index:]
        copy.transportation_names = self.transportation_names[index:]
        return copy
    
    def join(self, other: "TramConnection") -> "TramConnection":
        copy = TramConnection()
        copy.stops = self.stops + other.stops
        copy.stop_arrival_times = self.stop_arrival_times + other.stop_arrival_times
        copy.stop_departure_times = self.stop_departure_times + other.stop_departure_times
        copy.transportation_names = self.transportation_names + other.transportation_names
        return copy
    
    def __len__(self):
        return len(self.stops)
    
    @property
    def tram_name(self):
        return self.transportation_names[0]
    
    @property
    def arrival_times(self):
        return self.stop_arrival_times
    
    @property
    def departure_times(self):
        return self.stop_departure_times
    
    @property
    def arrival_time(self):
        return self.stop_arrival_times[0]
    
    @property
    def departure_time(self):
        return self.stop_departure_times[0]
    
    def __repr__(self):
        return f"TramConnection({self.stops}, {self.stop_arrival_times}, {self.stop_departure_times}, {self.transportation_names})"
    
    def __lt__(self, other):
        return False
    
    def __gt__(self, other):
        return True

class TramStop:

    def __repr__(self):
        return f"TramStop(name={self.name!r})"

    def __init__(self, gt_stop):
        self.gt_stop = gt_stop

        # first stop of every tramstop connection is
        # the stop itself with arrival and departure time
        self.connections = []

    def get_departure_times(self) -> list[datetime]:
        return [c.departure_times[0] for c in self.connections]
    
    def get_arrival_times(self) -> list[datetime]:
        return [c.arrival_times[0] for c in self.connections]
    
    def get_departures_after(self, time: datetime) -> list[TramConnection]:
        return [
            c for c in self.connections
            if c.departure_times[0] >= time
        ]

    def __eq__(self, value):
        if isinstance(value, TramStop):
            return self.name == value.name
        return self == value
    
    def __ne__(self, value):
        return not self.__eq__(value)
    
    def __hash__(self):
        return hash(self.name)
 
    @property
    def name(self):
        return self.gt_stop["stop_name"]
    
    @property
    def id(self):
        return self.gt_stop["stop_id"]
    
    @property
    def coords(self) -> set[float]:
        return (
            float(self.gt_stop["stop_lat"]),
            float(self.gt_stop["stop_lon"]),
        )
    
    def add_connection(self, connection):
        self.connections.append(connection)

    def sort_connections(self):
        self.connections = sorted(
            self.connections,
            key=lambda con: con.departure_time
        )
    
tram_stops_map = {}
def get_tram_stop(gt_stop) -> TramStop:
    stop_name = gt_stop["stop_name"]
    if stop_name not in tram_stops_map:
        tram_stops_map[stop_name] = TramStop(gt_stop)
    return tram_stops_map[stop_name]

def get_tram_stops_by_name(name) -> list[TramStop]:
    stops = []
    for stop in tram_stops_map.values():
        if name in stop.name:
            stops.append(stop)
    return stops

In [115]:
lookup_day = "20250121" # YYYYMMDD

gt_tram_routes = [
    route for route in gt_routes
    if route["route_type"] == GT_TRAM_STOP_TYPE
]

for gt_tram_route in gt_tram_routes:
    tram_route_name = gt_tram_route['route_short_name']
    print(f"Processing Tram Route #{tram_route_name}")

    if not gt_tram_route["route_short_name"].isnumeric():
        print("Aborting because of non-numeric name.")
        continue

    gt_tram_trips = gt_get_all(gt_trips, "route_id", gt_tram_route["route_id"])

    unique_stop_names = set()

    for gt_tram_trip in gt_tram_trips:
        if not is_trip_available(gt_tram_trip["service_id"], lookup_day):
            continue

        print(gt_tram_trip)

        gt_tram_trip_stop_times = sorted(
            gt_stop_times_map[gt_tram_trip["trip_id"]],
            key=lambda stop_time: int(stop_time["stop_sequence"])
        )

        gt_tram_trip_stops = [
            gt_stops_map[stop_time["stop_id"]]
            for stop_time in gt_tram_trip_stop_times
        ]

        # if it has a parent, select that one instead
        for i, stop in enumerate(gt_tram_trip_stops):
            if stop["parent_station"]:
                # rename parent because many parents are renamed to similar things
                # like "Bahnhof" which isn't good since I assume unique names
                parent_stop = gt_stops_map[stop["parent_station"]]
                if "has_been_renamed" not in parent_stop:
                    parent_stop["stop_name"] = stop["stop_name"]
                    parent_stop["has_been_renamed"] = True
                gt_tram_trip_stops[i] = parent_stop

        connection = TramConnection()
        for gt_stop_time, gt_stop in zip(gt_tram_trip_stop_times, gt_tram_trip_stops):
            tram_stop: TramStop = get_tram_stop(gt_stop)

            connection.add_stop(tram_stop,
                gt_parse_time(gt_stop_time["arrival_time"], lookup_day),
                gt_parse_time(gt_stop_time["departure_time"], lookup_day),
                tram_route_name)
        
        # if gt_tram_trip["direction_id"] == "0" or True:
        #     for i, (stop, dep) in enumerate(zip(connection.stops, connection.departure_times)):
        #         if "Kinkel" in stop.name:
        #             next_name = connection.stops[i + 1].name
        #             print(dep, f"{stop.name} -> {next_name}")

        for i, tram_stop in enumerate(connection.stops):
            connection_slice = connection.slice(i)
            if len(connection_slice.stops) > 1:
                tram_stop.add_connection(connection_slice)

    break

print(f"Found {len(tram_stops_map.values())} Tram Stops.")

print("Sorting Stop Connections...")
for tram_stop in tram_stops_map.values():
    tram_stop.sort_connections()

print("Finished!")

Processing Tram Route #10
{'route_id': '1-10-P-j25-1', 'service_id': 'T0', 'trip_id': '1.T0.1-10-P-j25-1.1.R', 'shape_id': '1-10-P-j25-1.1.R', 'trip_headsign': 'Zürich Flughafen, Fracht', 'direction_id': '1', 'block_id': ''}
{'route_id': '1-10-P-j25-1', 'service_id': 'T0+uf400', 'trip_id': '1001.T0.1-10-P-j25-1.9.H', 'shape_id': '1-10-P-j25-1.9.H', 'trip_headsign': 'Zürich, Bahnhofplatz/HB', 'direction_id': '0', 'block_id': ''}
{'route_id': '1-10-P-j25-1', 'service_id': 'T0+nw400', 'trip_id': '1002.T0.1-10-P-j25-1.9.H', 'shape_id': '1-10-P-j25-1.9.H', 'trip_headsign': 'Zürich, Bahnhofplatz/HB', 'direction_id': '0', 'block_id': ''}
{'route_id': '1-10-P-j25-1', 'service_id': 'T0+uf400', 'trip_id': '1005.T0.1-10-P-j25-1.9.H', 'shape_id': '1-10-P-j25-1.9.H', 'trip_headsign': 'Zürich, Bahnhofplatz/HB', 'direction_id': '0', 'block_id': ''}
{'route_id': '1-10-P-j25-1', 'service_id': 'T0+nw400', 'trip_id': '1006.T0.1-10-P-j25-1.9.H', 'shape_id': '1-10-P-j25-1.9.H', 'trip_headsign': 'Zürich, Ba

In [69]:
search_query = "Letzi"

time = gt_parse_time("12:00:00", lookup_day)
stop = get_tram_stops_by_name(search_query)[0]
departures = stop.get_departures_after(time)
first_departure = departures[0]

print(f"Next Departure from {stop.name} is at {first_departure.departure_time} with T{first_departure.tram_name}:")
print(first_departure)

Next Departure from Zürich, Letzistrasse is at 2025-01-21 12:00:18 with T9:
[2025-01-21 12:00:06] Zürich, Letzistrasse           (9) [2025-01-21 12:00:18]
[2025-01-21 12:00:54] Zürich, Kinkelstrasse          (9) [2025-01-21 12:01:06]
[2025-01-21 12:02:00] Zürich, Seilbahn Rigiblick     (9) [2025-01-21 12:02:18]
[2025-01-21 12:03:12] Zürich, Winkelriedstrasse      (9) [2025-01-21 12:03:24]
[2025-01-21 12:04:18] Zürich, Haldenbach             (9) [2025-01-21 12:04:30]
[2025-01-21 12:05:42] Zürich, ETH/Universitätsspital (9) [2025-01-21 12:06:00]
[2025-01-21 12:07:18] Zürich, Kantonsschule          (9) [2025-01-21 12:07:30]
[2025-01-21 12:08:48] Zürich, Kunsthaus              (9) [2025-01-21 12:09:06]
[2025-01-21 12:11:00] Zürich, Bellevue               (9) [2025-01-21 12:11:30]
[2025-01-21 12:13:06] Zürich, Bürkliplatz            (9) [2025-01-21 12:13:24]
[2025-01-21 12:14:36] Zürich, Kantonalbank           (9) [2025-01-21 12:14:54]
[2025-01-21 12:15:48] Zürich, Paradeplatz            (9

In [109]:
import bisect
MAX_TRANSITION_SECONDS = 30 * 60
MIN_CHANGE_BUFFER_SECONDS = 30

def calc_dijkstra_path_to(start_time: datetime, start: TramStop, destination_criterium) -> list[TramStop]:
    visited_stops = set()
    start_connection = TramConnection([start], [start_time], [None], [None])
    visit_stack = [(start_time, start_connection, start)]

    while len(visit_stack) > 0:
        curr_time, prev_stops, stop = visit_stack.pop(0)
        if stop in visited_stops:
            continue

        visited_stops.add(stop)

        connections = stop.get_departures_after(curr_time)
        for connection in connections:
            wait_seconds = (connection.arrival_time - curr_time).total_seconds()
            
            if len(prev_stops) >= 2:
                last_tram_name = prev_stops.transportation_names[-2]
                if last_tram_name is not None and connection.tram_name != last_tram_name:
                    # it's a change of tram!
                    if wait_seconds < MIN_CHANGE_BUFFER_SECONDS:
                        continue

            if wait_seconds >= MAX_TRANSITION_SECONDS:
                break

            new_stop = connection.stops[1]
            new_time = connection.arrival_times[1]
            if new_stop in visited_stops:
                continue

            prev_stops_copy = prev_stops.slice(0)
            prev_stops_copy.stop_departure_times[-1] = connection.departure_time
            prev_stops_copy.transportation_names[-1] = connection.tram_name

            prev_stops_copy.add_stop(new_stop, new_time, None, None)

            if destination_criterium(new_stop):
                return prev_stops_copy
            
            bisect.insort(visit_stack, (new_time, prev_stops_copy, new_stop))
            
    print(f"{visited_stops=}")
    raise Exception("Couldn't find connection in time")

def calc_dijkstra_path_between(start_time: datetime, start: TramStop, destination: TramStop) -> list[TramStop]:
    destination_criterium = lambda stop: stop == destination
    return calc_dijkstra_path_to(start_time, start, destination_criterium)

In [110]:
stop1 = get_tram_stops_by_name("Zoo")[0]
stop2 = get_tram_stops_by_name("Letzi")[0]
start_time = gt_parse_time("12:00:00", lookup_day)

connection = calc_dijkstra_path_between(start_time, stop1, stop2)
print(connection)

[2025-01-21 12:00:00] Zürich, Zoo                    (6) [2025-01-21 12:04:30]
[2025-01-21 12:05:54] Zürich, Susenbergstrasse       (6) [2025-01-21 12:06:06]
[2025-01-21 12:07:00] Zürich, Zürichbergstrasse      (6) [2025-01-21 12:07:12]
[2025-01-21 12:08:18] Zürich, Toblerplatz            (6) [2025-01-21 12:08:36]
[2025-01-21 12:09:54] Zürich, Kirche Fluntern        (6) [2025-01-21 12:10:12]
[2025-01-21 12:11:42] Zürich, Voltastrasse           (6) [2025-01-21 12:11:54]
[2025-01-21 12:13:24] Zürich, Platte                 (6) [2025-01-21 12:13:42]
[2025-01-21 12:15:54] Zürich, ETH/Universitätsspital (9) [2025-01-21 12:18:00]
[2025-01-21 12:18:42] Zürich, Haldenbach             (9) [2025-01-21 12:18:54]
[2025-01-21 12:19:42] Zürich, Winkelriedstrasse      (9) [2025-01-21 12:19:54]
[2025-01-21 12:20:48] Zürich, Seilbahn Rigiblick     (9) [2025-01-21 12:21:06]
[2025-01-21 12:21:54] Zürich, Kinkelstrasse          (9) [2025-01-21 12:22:06]
[2025-01-21 12:22:42] Zürich, Letzistrasse          

In [111]:
def calc_nearest_neighbor_search(start_stop: TramStop, start_time: datetime) -> TramConnection:
    unvisited_stops = set(tram_stops_map.values())
    unvisited_stops.remove(start_stop)
    destination_criterium = lambda stop: stop in unvisited_stops
    visit_count = 0

    curr_stop = start_stop
    curr_time = start_time
    path = TramConnection()
    print(f"Starting at {curr_stop.name}")
    while len(unvisited_stops) > 0:
        new_path = calc_dijkstra_path_to(curr_time, curr_stop, destination_criterium)
        curr_time = new_path.arrival_times[-1]

        if len(path) > 0:
            path.remove_last()

        visit_count += 1
        path = path.join(new_path)
        curr_stop = path.stops[-1]
        unvisited_stops.remove(curr_stop)


        print(f"[{visit_count}] {curr_time} -> {curr_stop.name}")

    return path

letzi = get_tram_stops_by_name("Letzi")[0]
start_time = gt_parse_time("10:00:00", lookup_day)

path = calc_nearest_neighbor_search(letzi, start_time)

Starting at Zürich, Letzistrasse
[1] 2025-01-21 10:00:54 -> Zürich, Kinkelstrasse
[2] 2025-01-21 10:02:00 -> Zürich, Seilbahn Rigiblick
[3] 2025-01-21 10:03:12 -> Zürich, Winkelriedstrasse
[4] 2025-01-21 10:04:18 -> Zürich, Haldenbach
[5] 2025-01-21 10:05:42 -> Zürich, ETH/Universitätsspital
[6] 2025-01-21 10:07:18 -> Zürich, Kantonsschule
[7] 2025-01-21 10:08:48 -> Zürich, Kunsthaus
[8] 2025-01-21 10:11:00 -> Zürich, Bellevue
[9] 2025-01-21 10:12:42 -> Zürich, Helmhaus
[10] 2025-01-21 10:13:48 -> Zürich, Rathaus
[11] 2025-01-21 10:14:48 -> Zürich, Rudolf-Brun-Brücke
[12] 2025-01-21 10:16:12 -> Zürich, Central
[13] 2025-01-21 10:18:24 -> Zürich, Bahnhofquai/HB
[14] 2025-01-21 10:20:36 -> Zürich, Sihlquai/HB
[15] 2025-01-21 10:22:00 -> Zürich, Museum für Gestaltung
[16] 2025-01-21 10:23:06 -> Zürich, Limmatplatz
[17] 2025-01-21 10:24:18 -> Zürich, Quellenstrasse
[18] 2025-01-21 10:25:12 -> Zürich, Löwenbräu
[19] 2025-01-21 10:26:12 -> Zürich, Escher-Wyss-Platz
[20] 2025-01-21 10:28:00 -

In [112]:
print(path)

[2025-01-21 10:00:00] Zürich, Letzistrasse           (9) [2025-01-21 10:00:18]
[2025-01-21 10:00:54] Zürich, Kinkelstrasse          (9) [2025-01-21 10:01:06]
[2025-01-21 10:02:00] Zürich, Seilbahn Rigiblick     (9) [2025-01-21 10:02:18]
[2025-01-21 10:03:12] Zürich, Winkelriedstrasse      (9) [2025-01-21 10:03:24]
[2025-01-21 10:04:18] Zürich, Haldenbach             (9) [2025-01-21 10:04:30]
[2025-01-21 10:05:42] Zürich, ETH/Universitätsspital (9) [2025-01-21 10:06:00]
[2025-01-21 10:07:18] Zürich, Kantonsschule          (9) [2025-01-21 10:07:30]
[2025-01-21 10:08:48] Zürich, Kunsthaus              (9) [2025-01-21 10:09:06]
[2025-01-21 10:11:00] Zürich, Bellevue               (4) [2025-01-21 10:11:24]
[2025-01-21 10:12:42] Zürich, Helmhaus               (4) [2025-01-21 10:13:00]
[2025-01-21 10:13:48] Zürich, Rathaus                (4) [2025-01-21 10:14:06]
[2025-01-21 10:14:48] Zürich, Rudolf-Brun-Brücke     (4) [2025-01-21 10:15:06]
[2025-01-21 10:16:12] Zürich, Central               

In [113]:
from PIL import Image, ImageDraw, ImageFont

card_coords = {
    "karte1": [(47.401543, 8.483763), (47.339615, 8.588987)],
    "karte2": [(47.459360, 8.437810), (47.310365, 8.637404)],
    "karte3": [(47.459360, 8.437810), (47.310365, 8.637404)]
}

def visualize_tramconnections(stops, card="karte3"):
    top_left_coord, bottom_right_coord = card_coords[card]

    def coordinate_to_01_pos(coord):
        coord_deltas = (
            top_left_coord[0] - bottom_right_coord[0],
            bottom_right_coord[1] - top_left_coord[1]
        )
        return (
            1 - (coord[0] - bottom_right_coord[0]) / coord_deltas[0],
            (coord[1] - top_left_coord[1]) / coord_deltas[1]
        )

    image: Image = Image.open(f"assets/{card}.png")
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    stop_visit_counts = {}

    previous_point = None
    for index, stop in enumerate(stops):
        coord = stop.coords
        relative_pos = coordinate_to_01_pos(coord)
        x, y = relative_pos[1] * image.size[0], relative_pos[0] * image.size[1]

        if previous_point is not None:
            draw.line([previous_point, (x, y)], fill="red", width=5)

        previous_point = (x, y)

    for index, stop in enumerate(stops):
        if stop in stop_visit_counts:
            stop_visit_counts[stop] += 1
        else:
            stop_visit_counts[stop] = 1
        visit_count = stop_visit_counts[stop]

        coord = stop.coords
        relative_pos = coordinate_to_01_pos(coord)
        x, y = relative_pos[1] * image.size[0], relative_pos[0] * image.size[1]
        radius = 20
        if visit_count == 1:
            draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill="red")

        draw.text((x - 5, y - 25 + visit_count * 10), str(index), fill="black", font=font)

    image.show()

visualize_tramconnections(path.stops)

In [58]:
for stop in tram_stops_map.values():
    # number of trams at this stop
    reachability = len(set([n for c in stop.connections for n in c.transportation_names]))

    stop.heuristics = {
        "reachability": reachability
    }

print(path.stops[0].heuristics)

{'reachability': 5}


In [64]:
[c.arrival_time for c in path.stops[0].connections if c.tram_name == "5"]

[datetime.datetime(2025, 1, 19, 5, 7, 24),
 datetime.datetime(2025, 1, 19, 5, 22, 24),
 datetime.datetime(2025, 1, 19, 5, 24, 12),
 datetime.datetime(2025, 1, 19, 5, 39, 12),
 datetime.datetime(2025, 1, 19, 8, 16, 12),
 datetime.datetime(2025, 1, 19, 8, 31, 12),
 datetime.datetime(2025, 1, 19, 9, 29, 6),
 datetime.datetime(2025, 1, 19, 9, 58, 24),
 datetime.datetime(2025, 1, 19, 10, 2, 30),
 datetime.datetime(2025, 1, 19, 18, 21, 6),
 datetime.datetime(2025, 1, 19, 18, 26, 30),
 datetime.datetime(2025, 1, 19, 22, 46, 48),
 datetime.datetime(2025, 1, 19, 23, 16, 48),
 datetime.datetime(2025, 1, 20, 0, 38, 18),
 datetime.datetime(2025, 1, 20, 0, 43, 48)]